In [ ]:
import nltk
import os
from glob import glob
import json
import emoji
from collections import defaultdict
from tqdm import tqdm
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
root_path = os.path.dirname(os.getcwd())
data_path = os.path.join(root_path, "data\\mention_network_chats\\")
data_path


In [ ]:
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

twitch_system_words = { # Common Twitch system messages
    'welcome', 'joined', 'left', 'hosted', 'hosting',
    'subscribed', 'gifted', 'sub', 'prime', 'tier', 
    'months', 'streak', 'resubscribed', 'raided'
}

stopwords += list(twitch_system_words)

chat_filler_stopwords = {
    'lol', 'lmao', 'like', 'hai', 'mhm', 'yeah', 'yawn', 
    'bro', 'back', 'get', 'one', 'good', 'want', 'think', 
    'know', 'im', 'thats', 'dont', 'cant', 'tho', 'oh', 'uh',
    'ok', 'okay', 'right', 'maybe', 'well', 'see', 'bot',
    'watch', 'streams', 'turn', 'stop', 'please', 'got', 'ass', 'fire', 'call',
    'kekw', 'lul', 'pog', 'poggers', 'monka', 'ez', 'monkas', 'pepe', 'xd', 
    'omegalul', 'gasm', 'hi'

}

stopwords += list(chat_filler_stopwords)

# Define your spam phrases globally outside the functions
# Add any suspected bot/alert phrases confirmed from your investigation
SPAM_PHRASES = {
    'subscribed with prime', 
    'subscribed for',
    'just subscribed',
    'gifted a subscription', 
    'gifted a sub',
    'gifted sub',
    'gift',
    'donate',
    'consecutive streams'
}


def generate_alias_set(streamer_name):
    """
    Generates a set of potential aliases by simplifying the complex streamer name.
    """
    aliases = {streamer_name.lower()}
    
    # 1. Remove leading/trailing numbers and symbols (2xrakai -> rakai)
    # This targets names like 'Lirik123' or 'XqcOW'
    simple_name = re.sub(r'(\d+|tv|live|gaming|ow|hd|_|-)$', '', streamer_name, flags=re.IGNORECASE)
    simple_name = re.sub(r'^(\d+|tv|live|gaming|ow|hd|_|-)', '', simple_name, flags=re.IGNORECASE)

    if simple_name.lower() != streamer_name.lower() and len(simple_name) >= 3:
        aliases.add(simple_name.lower())
    
    # 2. Generate prefixes of the simplified name (rakai -> rak, raka, rakai)
    # This catches the original 'aceu' -> 'ace'
    if len(simple_name) >= 3:
        for i in range(3, len(simple_name) + 1):
            aliases.add(simple_name[:i].lower())
            
    # 3. Heuristic for complex names: The common short alias is often the last part.
    # This specifically targets 'rakai' -> 'kai'
    if len(simple_name) > 4:
        short_alias = simple_name[-3:].lower() # Take the last 3 characters
        if short_alias.isalpha(): # Only include if it's not just numbers/symbols
            aliases.add(short_alias)
            
    return aliases


def is_auto_message(text):
    """
    Checks if a message body contains known system/alert phrases.
    """
    lowered_text = text.lower()
    
    # Check for known system phrases
    for phrase in SPAM_PHRASES:
        if phrase in lowered_text:
            return True
            
    # Optional: Check for short, common non-human phrases 
    # (e.g., just 'hi' or 'xD' might pass this, but you can adjust)
    # if len(lowered_text.split()) < 2:
    #    return True 

    return False

def squash_spam(token):
    
    if len(token) > 2:
        # Check for 3 or more consecutive identical characters
        pattern = r'(.)\1{2,}'
        if re.search(pattern, token):
            return True


def remove_mentions(text):
    """
    Removes all user mentions (e.g., @MukkingAround, @OmegaTooYew) 
    from the text, replacing them with a space.
    """
    # Regex pattern: Looks for the '@' symbol followed by one or more 
    # non-whitespace characters, up to the end of the word/line.
    # It handles common Twitch names (which often include letters, numbers, and underscores).
    pattern = r'@\S+'
    return re.sub(pattern, ' ', text).strip()


def remove_streamer_name(text, streamer):
    if not streamer:
        return text

    aliases_to_remove = generate_alias_set(streamer)

    # Escape and sort by length descending
    escaped_aliases = [re.escape(alias) for alias in aliases_to_remove]
    escaped_aliases.sort(key=len, reverse=True)
    
    pattern_str = '|'.join(escaped_aliases)
    
    # Simple, high-impact substitution:
    cleaned_text = re.sub(pattern_str, " ", text, flags=re.IGNORECASE) 
    
    return cleaned_text

def normalize_urls(text):
    url_pattern = r'https?://\S+|www\.\S+'
    return re.sub(url_pattern, '', text)

def preprocess_text(text, streamer):

    text = normalize_urls(text)
    text = remove_mentions(text)
    text = remove_streamer_name(text, streamer)
    text = emoji.demojize(text, delimiters=(" ", " ")) # Reformat emojis to text
    lowered_text = text.lower()
    tokens = nltk.word_tokenize(lowered_text)
    tokens = [token for token in tokens if token.isalnum()] # Keep only alphanumeric tokens
    tokens = [token for token in tokens if not squash_spam(token)]
    tokens = [token for token in tokens if token not in stopwords]
    return tokens

In [ ]:
# Get all chat files
chat_files = list(glob(data_path + '*_chat.json'))
tokenized_chat = defaultdict(list)
print(f"Found {len(chat_files)} chat files to analyze\n")

for i, chat_file in tqdm(enumerate(chat_files), total=len(chat_files)):
    # Extract streamer name from filename
    filename = chat_file.split("\\")[-1].split(".")[0]  # Remove .json
    source_streamer = filename.split('_')[0].lower()  # First part is streamer name
    
    try:
        with open(chat_file, 'r', encoding='utf-8') as f:
            chat_data = json.load(f)       
        comments = chat_data.get('comments', [])
        for comment in tqdm(comments, desc=f"Processing comments for {source_streamer}"):
            fragments = comment['message']['fragments']
            if not fragments:
                continue
            if comment['message']['fragments'][0]['emoticon']:
                continue
            text = comment['message']['fragments'][0]['text']
            if is_auto_message(text):
                continue  # Skip auto messages
            tokens = preprocess_text(text, source_streamer)
            tokenized_chat[source_streamer] += tokens
        if i == 100:
            break
    except Exception as e:
        print(f"[{i}/{len(chat_files)}] {source_streamer:15s}: Error - {e}")


In [ ]:
streamer_comment = dict(tokenized_chat)

In [ ]:
streamer_map = {i: streamer for i, streamer in enumerate(list(streamer_comment.keys()))} 

In [ ]:
streamer_chats = [" ".join(chat) for chat in list(streamer_comment.values())]

In [ ]:

tfidf_vectorizer = TfidfVectorizer()  

tfidf = tfidf_vectorizer.fit_transform(streamer_chats)

words = tfidf_vectorizer.get_feature_names_out()

In [ ]:
# streamer_map

In [ ]:
def show_wordcloud(streamer, tfidf_vector):
    scores = tfidf_vector.toarray()[0]
    word_scores = {words[i]: scores[i] for i in range(len(words))}
    
    wc = WordCloud(width=800, height=400).generate_from_frequencies(word_scores)
    
    plt.imshow(wc)
    plt.axis("off")
    plt.title(streamer)
    plt.show()
for idx in range(len(streamer_comment)):
    show_wordcloud(streamer_map[idx], tfidf[idx])